## CLoud Infrastructure Assignment 2

## Question 1

In [1]:
from itertools import combinations

In [30]:
# Check intersection of quorums = 1 proces
def check_intersection(target,c):
    status = True
    for t in target:
        if not t.intersection(c):
            status = False
            break
        if len(t.intersection(c)) > 1:
            status = False
            break
    return status

def validate_quorums(processes):
    for key in processes.keys():
        status = check(key, processes)
        if status: 
            print(f"Validation successful for {key}")

        
def check(process_id, processes):
    status = True
    keys = processes.keys()
    quorum = set(processes.get(process_id))
    for key in keys:
        if key == process_id:
            continue
        q = set(processes.get(key))
        if not quorum.intersection(q):
            print(f"Validation failed for processes {key} and {process_id}")
            status = False
    return status
        

In [34]:
# Generate all the combinations for nCk
def generateProcessQuorumCombinations(n,k):
    combs = combinations([i for i in range(1,n+1)],k)
    combs = list(combs)
    combs = [set(comb) for comb in combs]

    i = 1
    target = [combs.pop(0)]
    for comb in combs:
        if i <= n:
            if check_intersection(target, comb):
                target.append(comb)
                i+=1
    return target

def MaekawasQuorums(n, k):
    target = generateProcessQuorumCombinations(n,k)
    quorums = target.copy()
    missing = []
    process_quorum = {}
    for i in range(1,n+1):
        status = False
        for q in quorums:
            if (i in q):
                process_quorum[i] = q
                quorums.remove(q)
                status = True
                break
        if not status:
            #print(f"process {i} missing")
            missing.append(i)
    if missing:
        for miss in missing:
            for q in quorums:
                for p in q:
                    if (miss in process_quorum.get(p)):
                        #print(f"Swapping {miss} with {p}")
                        process_quorum[miss] = process_quorum[p]
                        process_quorum[p] = q
                        quorums.remove(q)
                        missing.remove(miss)
                        break
    return process_quorum     

### (n,k) pairs as per the algorithm
- (3,2)
- (7,3)
- (13,4)
- (21,5)

### Create the Quorums

In [38]:
process_quorum = MaekawasQuorums(21,5)
process_quorum

{1: {1, 2, 3, 4, 5},
 2: {2, 6, 10, 14, 18},
 3: {3, 6, 11, 16, 21},
 4: {4, 6, 12, 17, 19},
 5: {5, 8, 11, 17, 18},
 6: {1, 6, 7, 8, 9},
 7: {2, 7, 11, 15, 19},
 8: {2, 8, 12, 16, 20},
 9: {2, 9, 13, 17, 21},
 10: {1, 10, 11, 12, 13},
 11: {4, 9, 11, 14, 20},
 12: {3, 9, 12, 15, 18},
 13: {3, 8, 13, 14, 19},
 14: {1, 14, 15, 16, 17},
 15: {4, 8, 10, 15, 21},
 16: {4, 7, 13, 16, 18},
 17: {3, 7, 10, 17, 20},
 18: {1, 18, 19, 20, 21},
 19: {5, 9, 10, 16, 19},
 21: {5, 7, 12, 14, 21},
 20: {5, 6, 13, 15, 20}}

### Validate the quorums
- Each quorum has intersection with all other quorums

In [37]:
validate_quorums(process_quorum)

Validation successful for 1
Validation successful for 2
Validation successful for 3
Validation successful for 4
Validation successful for 5
Validation successful for 6
Validation successful for 7
Validation successful for 8
Validation successful for 9
Validation successful for 10
Validation successful for 11
Validation successful for 12
Validation successful for 13


### Q1-(b) Mutual Exlusion in Distributed systems

In [50]:
import threading
import time
from collections import deque

class CriticalSection:
    process_quorum_dict = {}
    process_queue = {}
    
    def __init__(self, process_quorum):
        self.process_quorum_dict = process_quorum
        self._lock = threading.Lock()
        for k in process_quorum.keys():
            self.process_queue[k] = deque([])
        
    def enter_cs(self, pid):
        retries,count = 5,0
        self.request_quorum(pid)
       
        if self.get_quorum_reply(pid):
            with self._lock:
                print(f"[INFO] Process {pid} entering critical section!")
                time.sleep(1)
            
    def request_quorum(self,pid):
        print(f"[INFO] Process {pid} requested access to quorum {self.process_quorum_dict.get(pid)}")
        quorum = self.process_quorum_dict.get(pid)
        for proc in quorum:
            self.process_queue.get(proc).append([pid,"REQUEST"])
    
    def get_quorum_reply(self,pid):
        quorum = self.process_quorum_dict.get(pid)
        quorum_len = len(quorum)
        count = 0
        for proc in quorum:
            p_q =self.process_queue.get(proc)
            if p_q:
                elem = p_q[0]
                if elem[0] == pid:
                    elem[1] ="VOTED"
                    count +=1
        if count == quorum_len:
            for proc in quorum:
                self.process_queue.get(proc).popleft()
            print(f"[INFO] Quorum granted access to process {pid}")
            return True
        
        return False

### Critical section mutual exclusion

In [51]:
cs = CriticalSection(process_quorum)
threads = []
for p in process_quorum.keys():
    t = threading.Thread(target=cs.enter_cs, args=(p,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()
print("[INFO] Main thread ends")

[INFO] Process 1 requested access to quorum {1, 2, 3, 4, 5}
[INFO] Quorum granted access to process 1
[INFO] Process 1 entering critical section!
[INFO] Process 2 requested access to quorum {2, 6, 10, 14, 18}
[INFO] Quorum granted access to process 2
[INFO] Process 3 requested access to quorum {3, 6, 11, 16, 21}
[INFO] Process 4 requested access to quorum {4, 6, 12, 17, 19}[INFO] Quorum granted access to process 3

[INFO] Quorum granted access to process 4
[INFO] Process 5 requested access to quorum {5, 8, 11, 17, 18}
[INFO] Process 6 requested access to quorum {1, 6, 7, 8, 9}
[INFO] Quorum granted access to process 5
[INFO] Quorum granted access to process 6[INFO] Process 7 requested access to quorum {2, 7, 11, 15, 19}
[INFO] Process 8 requested access to quorum {2, 8, 12, 16, 20}
[INFO] Quorum granted access to process 8

[INFO] Quorum granted access to process 7[INFO] Process 9 requested access to quorum {2, 9, 13, 17, 21}

[INFO] Quorum granted access to process 9
[INFO] Process 10

## Question 2

In [14]:
import pandas as pd
import numpy as np

In [15]:
def MinMinAlgorithm(df):
    scheduling = df.copy()
    
    cols = scheduling.columns
    iters = len(scheduling.index)
    while iters > 0:
        index = scheduling.index
        min_time = scheduling.at[index[0],cols[0]]
        min_time_m = cols[0]
        min_index = index[0]
        for col in cols:

            for ind in index:
                task_time = scheduling.at[ind,col]
                if task_time < min_time:
                    min_time = task_time
                    min_time_m = col
                    min_index = ind
        print(f"Assign Task: {min_index} to machine: {min_time_m} with time = {min_time}")
        scheduling.drop(min_index, axis=0, inplace=True)
        scheduling[min_time_m] = scheduling[min_time_m].apply(lambda value: value+min_time)
        iters -=1
        
def MinMaxAlgorithm(df):
    scheduling = df.copy()
    
    cols = scheduling.columns
    iters = len(scheduling.index)
    while iters > 0:
        index = scheduling.index
        min_time_list = []
        max_time_sorted = []
        for ind in index:
            min_time = scheduling.at[ind,cols[0]]
            min_time_m = cols[0]
            min_index = ind
            for col in cols:
                if scheduling.at[ind,col] < min_time:
                    min_time = scheduling.at[ind,col]
                    min_time_m = col
                    min_index = ind
            min_time_list.append((min_index, min_time_m, min_time))
        max_time_sorted = sorted(min_time_list,key = lambda k: k[2], reverse=True)
       
        print(f"Assign Task: {max_time_sorted[0][0]} to machine {max_time_sorted[0][1]} with time = {max_time_sorted[0][2]}")
        scheduling.drop(max_time_sorted[0][0], axis=0, inplace=True)
        scheduling[max_time_sorted[0][1]] = scheduling[max_time_sorted[0][1]].apply(lambda value: value+max_time_sorted[0][2])
        #print(scheduling)
        iters -=1

### Read the Q2 csv file

In [47]:
hw2q2 = pd.read_csv("hw2_q2.csv", index_col="TaskList")

In [48]:
hw2q2

,M1,M2,M3,M4,M5,M6
TaskList,,,,,,
0,13,23,71,60,27,2
1,31,14,94,60,61,57
2,17,-,23,36,8,86
3,19,10,4,58,73,40
4,94,-,58,-,68,46
5,8,3,32,4,94,89
6,10,13,1,92,75,29
7,80,38,40,66,25,88


#### Replace the '-' by a large number 999999

In [21]:
ddf = hw2q2.replace(r'-',999999,regex=True)
# Convert the df to int64 data type
ddf = ddf.astype('int64')
ddf

,M1,M2,M3,M4,M5,M6
TaskList,,,,,,
0,13,23,71,60,27,2
1,31,14,94,60,61,57
2,17,999999,23,36,8,86
3,19,10,4,58,73,40
4,94,999999,58,999999,68,46
5,8,3,32,4,94,89
6,10,13,1,92,75,29
7,80,38,40,66,25,88


### MinMinAlgorithm

In [22]:
MinMinAlgorithm(ddf)

Assign Task: 6 to machine: M3 with time = 1
Assign Task: 0 to machine: M6 with time = 2
Assign Task: 5 to machine: M2 with time = 3
Assign Task: 3 to machine: M3 with time = 5
Assign Task: 2 to machine: M5 with time = 8
Assign Task: 1 to machine: M2 with time = 17
Assign Task: 7 to machine: M5 with time = 33
Assign Task: 4 to machine: M6 with time = 48


### MinMaxAlgorithm

In [23]:
MinMaxAlgorithm(ddf)

Assign Task: 4 to machine M6 with time = 46
Assign Task: 7 to machine M5 with time = 25
Assign Task: 2 to machine M1 with time = 17
Assign Task: 0 to machine M2 with time = 23
Assign Task: 1 to machine M2 with time = 37
Assign Task: 3 to machine M3 with time = 4
Assign Task: 6 to machine M3 with time = 5
Assign Task: 5 to machine M4 with time = 4
